In [14]:
##COPYRIGHT from 仲益教育

##Part 1: Data Exploration
#0.0: import packages
#pandas is a data structure package
#seaborn is a statistial data visualization
#matplotlib is a plotting package
#numpy is a scientific calculation in Python
#sklearn is a data science package
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats
import sys

#0.1: import data
#change to your local path
train_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/train.csv")
test_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/test.csv")
train_data=test_data
#1.1: look at what's in the data
train_data.columns
#look at data description, look at variable table
#or use excel to look at the data
#or use the following useful functions
train_data.shape
train_data.head()
train_data.tail()
#Change dataframe index to Id
train_data.index=train_data["Id"]
train_data.drop("Id", axis=1, inplace=True)

# #2.1: look at what do we need to predict first
# #decriptive summary, look at the distribution of data
# train_data["SalePrice"].describe()
# #skewness and kurtosis
# print("Skewness: %f" % train_data['SalePrice'].skew())
# print("Kurtosis: %f" % train_data['SalePrice'].kurt())
# #histogram
# sns.distplot(train_data['SalePrice'])

# #2.2: look at what are the variables
# #select some variables you think are relevant
# #2.2.1 numerical variables: try LotArea (lot size in square feet) and year built
# #scatter plot
# var = 'LotArea'
# data = pd.concat([train_data['SalePrice'], train_data[var]], axis=1)
# data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));
# var = 'YearBuilt'
# data = pd.concat([train_data['SalePrice'], train_data[var]], axis=1)
# data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));
# #2.2.2 categorical variables: try OverQual (Overall material and finish quality)
# #boxplot
# var = 'OverallQual'
# data = pd.concat([train_data['SalePrice'], train_data[var]], axis=1)
# f, ax = plt.subplots(figsize=(8, 6))
# fig = sns.boxplot(x=var, y="SalePrice", data=data)
# fig.axis(ymin=0, ymax=800000);
# #2.2.3 correlation
# #heat map
# corrmat = train_data.corr()
# f, ax = plt.subplots(figsize=(12, 9))
# sns.heatmap(corrmat, vmax=.8, square=True);
# #top correlated variables
# k = 10 #number of variables for heatmap
# cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
# cm = np.corrcoef(train_data[cols].values.T)
# sns.set(font_scale=1.25)
# hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, 
#                  yticklabels=cols.values, xticklabels=cols.values)
# plt.show()
# #scatter plot 
# sns.set()
# cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF']
# sns.pairplot(train_data[cols], height = 2.5)
# plt.show();

#3.1 data cleaning: no correct answer
# how many % of data are missing
total = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print("how many data has missing value = " + str(sum(missing_data['Total'] >= 1)))
#3.1.1 fill missing value with mode
train_data["Electrical"]=train_data["Electrical"].fillna(train_data["Electrical"].mode()[0])
#3.1.2 fill missing value with 0
train_data["MasVnrArea"]=train_data["MasVnrArea"].fillna(0)
#3.1.3 OR fill missing value with None
train_data["MasVnrArea"]=train_data["MasVnrArea"].fillna("None")
#3.1.4 OR fill missing value with median
train_data["MasVnrArea"]=train_data["MasVnrArea"].fillna(train_data["MasVnrArea"].median())

#3.1.5 #dealing with missing data
#need to understand
total = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
train_data = train_data.drop((missing_data[missing_data['Total'] >= 1]).index,1)
#just checking that there's no missing data missing
train_data.isnull().sum().max() 

# #3.2 outliers
# #z-score
# saleprice_scaled = StandardScaler().fit_transform(train_data['SalePrice'][:,np.newaxis]);
# low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
# high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
# print('outer range (low) of the distribution:')
# print(low_range)
# print('\nouter range (high) of the distribution:')
# print(high_range)
# #scatter chart
# var = 'GrLivArea'
# data = pd.concat([train_data['SalePrice'], train_data[var]], axis=1)
# data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));
# #3.2.1 deleting points
# train_data.sort_values(by = 'GrLivArea', ascending = False)[:2]
# # train_data = train_data.drop(1299)
# # train_data = train_data.drop(524)
# var = 'TotalBsmtSF'
# data = pd.concat([train_data['SalePrice'], train_data[var]], axis=1)
# data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));
# #3.2.2 log transformation
# #histogram and normal probability plot
# print("Skewness: %f" % train_data['SalePrice'].skew())
# print("Kurtosis: %f" % train_data['SalePrice'].kurt())
# sns.distplot(train_data['SalePrice'], fit=norm);
# fig = plt.figure()
# #qq plot
# res = stats.probplot(train_data['SalePrice'], plot=plt)
# #applying log transformation
# train_data['SalePrice'] = np.log(train_data['SalePrice'])
# print("Skewness: %f" % train_data['SalePrice'].skew())
# print("Kurtosis: %f" % train_data['SalePrice'].kurt())
# sns.distplot(train_data['SalePrice'], fit=norm);
# fig = plt.figure()
# #qq plot
# res = stats.probplot(train_data['SalePrice'], plot=plt)

#3.3 convert numerical variables to categorical variables
train_data = train_data.replace({"MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                                       50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                                       80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                                       150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
                       "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                                   7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
                      })

#export data
train_data.to_csv("tes_clean_teacher.csv")

how many data has missing value = 33


In [64]:
##COPYRIGHT from 仲益教育
import sys
sys.path.append("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/May19thVer")
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from Help_functions_v2 import sklearn_Pvalue, sklearn_adjR2, RMSE
import numpy as np

#1: import data
train_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/May19thVer/train_clean_teacher.csv")
test_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/May19thVer/tes_clean_teacher.csv")
#2: feature engineering
#2.1: take a closer look at the categorical features before changing to dummy
categorical_features = train_data.select_dtypes(include = ["object"]).columns
train_cat = train_data[categorical_features]
#see what most category data are the same
#delete highly skew category data
pct=[]
for ix in train_cat.columns:
    temp=train_cat[ix].describe()
    pct.append(temp["freq"]/temp["count"])
skewData=pd.DataFrame(pct,index=train_cat.columns,columns=["skewness"])
skewData=skewData.sort_values(by="skewness",ascending=False)
print (skewData)
train_data = train_data.drop((skewData[skewData['skewness'] >= 0.95]).index,1) 
# Sale_Price=train_data.iloc[:,80]
# train_data=train_data.drop(["SalePrice"],axis=1)
data= pd.concat([train_data,test_data], keys=['x', 'y'])#here X is training data and Y testing data
# data=data.drop(["Id"],axis=1)
#2.2: some categorical features when there is information in the order
#Alley: Type of alley access to property
train_data=data
train_data=train_data.replace({"Alley":{"Grvl" : 1, "Pave" : 2}})
#BsmtCond: Evaluates the general condition of the basement
train_data=train_data.replace({"BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#BsmtExposure: Refers to walkout or garden level walls
train_data=train_data.replace({"BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3}})
#BsmtFinType1: Rating of basement finished area
train_data=train_data.replace({"BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6}})
#BsmtFinType2: Rating of basement finished area (if multiple types)
train_data=train_data.replace({"BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6}})
#BsmtQual: Evaluates the height of the basement
train_data=train_data.replace({"BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
#ExterCond: Evaluates the present condition of the material on the exterior
train_data=train_data.replace({"ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#ExterQual: Evaluates the quality of the material on the exterior 
train_data=train_data.replace({"ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5}})
#FireplaceQu: Fireplace quality
train_data=train_data.replace({"FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#Functional: Home functionality (Assume typical unless deductions are warranted)
train_data=train_data.replace({"Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8}})
#HeatingQC: Heating quality and condition
train_data=train_data.replace({"HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#KitchenQual: Kitchen quality
train_data=train_data.replace({"KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5}})
#LandSlope: Slope of property
train_data=train_data.replace({"LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3}})
#LotShape: General shape of property
train_data=train_data.replace({"LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4}})
#PavedDrive: Paved driveway
train_data=train_data.replace({"PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2}})

#2.3: Simplifications of existing categorical 
train_data["Condition1"] = train_data.Condition1.replace({"RRNe" : "Other", 
                                                  "RRNn" : "Other","PosA" : "Other", 
                                                   "RRAe" : "Other"
                                                  })
train_data["Electrical"] = train_data.Electrical.replace({"Mix" : "Other", 
                                                  "FuseP" : "Other"
                                                  })
train_data["Exterior1st"] = train_data.Exterior1st.replace({"AsphShn" : "Other", 
                                                  "CBlock" : "Other","ImStucc" : "Other", 
                                                   "BrkComm" : "Other","Stone" : "Other"
                                                  })
train_data["Exterior2nd"] = train_data.Exterior2nd.replace({"CBlock" : "Other", 
                                                  "AsphShn" : "Other","Stone" : "Other", 
                                                   "Brk Cmn" : "Other","ImStucc" : "Other"
                                                  })
train_data["Foundation"] = train_data.Foundation.replace({"Wood" : "Other", 
                                                  "Stone" : "Other"
                                                  })
# train_data["GarageType"] = train_data.GarageType.replace({"2Types" : "Other", 
#                                                   "CarPort" : "Other"
#                                                   })
train_data["HouseStyle"] = train_data.HouseStyle.replace({"2.5Fin" : "Other", 
                                                  "2.5Unf" : "Other",
                                                  "1.5Unf" : "Other"
                                                  })
train_data["LotConfig"] = train_data.LotConfig.replace({"FR3" : "FR2"
                                                  })
train_data["MSSubClass"] = train_data.MSSubClass.replace({"SC40" : "Other", 
                                                  "SC180" : "Other",
                                                  "SC45" : "Other",
                                                  "SC75" : "Other"
                                                  })
train_data["MSZoning"] = train_data.MSZoning.replace({"C (all)" : "Other", 
                                                  "RH" : "Other"
                                                  })
train_data["Neighborhood"] = train_data.Neighborhood.replace({"Blueste" : "Other", 
                                                  "NPkVill" : "Other",
                                                  "Veenker" : "Other"
                                                  })
train_data["RoofStyle"] = train_data.RoofStyle.replace({"Shed" : "Other", 
                                                  "Mansard" : "Other",
                                                  "Gambrel" : "Other",
                                                  "Flat" : "Other"
                                                  })
train_data["SaleCondition"] = train_data.SaleCondition.replace({"AdjLand" : "Other", 
                                                  "Alloca" : "Other"
                                                  })
train_data["SaleType"] = train_data.SaleType.replace({"Con" : "Other", 
                                                  "Oth" : "Other",
                                                  "CWD" : "Other", "ConLI" : "Other",
                                                  "ConLw" : "Other","ConLD" : "Other"
                                                  })

#2.4 Combinations of existing features
# Overall quality of the house
train_data["OverallGrade"] = train_data["OverallQual"] * train_data["OverallCond"]
# Overall quality of the exterior
train_data["ExterGrade"] = train_data["ExterQual"] * train_data["ExterCond"]
# Overall kitchen score
train_data["KitchenScore"] = train_data["KitchenAbvGr"] * train_data["KitchenQual"]
# Total number of bathrooms
train_data["TotalBath"] = train_data["BsmtFullBath"] + (0.5 * train_data["BsmtHalfBath"]) + \
train_data["FullBath"] + (0.5 * train_data["HalfBath"])
train_data.drop(["BsmtFullBath","BsmtHalfBath","FullBath","HalfBath"],axis=1)
# Total SF for house (incl. basement)
train_data["AllSF"] = train_data["GrLivArea"] + train_data["TotalBsmtSF"]
train_data.drop(["GrLivArea","TotalBsmtSF"],axis=1)
# Total SF for 1st + 2nd floors
train_data["AllFlrsSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"]
train_data.drop(["1stFlrSF","2ndFlrSF"],axis=1)
# Total SF for porch
train_data["AllPorchSF"] = train_data["OpenPorchSF"] + train_data["EnclosedPorch"] + \
train_data["3SsnPorch"] + train_data["ScreenPorch"]
train_data.drop(["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"],axis=1)
# House completed before sale or not
train_data["BoughtOffPlan"] = train_data.SaleCondition.replace({"Abnorml" : 0, "Alloca" : 0, "AdjLand" : 0, 
                                                      "Family" : 0, "Normal" : 0, "Partial" : 1})

#2.5 Polinomial transformation (Box-Cox)
# X^2, X^3, X^0.5, 1/X, Log(X)
# Find most important features relative to target
# print("Find most important features relative to target")
# corr = train_data.corr()
# corr.sort_values(["SalePrice"], ascending = False, inplace = True)
# print(corr.SalePrice)
train_data["OverallQual-s2"] = train_data["OverallQual"] ** 2
train_data["AllSF-2"] = train_data["AllSF"] ** 2
train_data["AllFlrsSF-2"] = train_data["AllFlrsSF"] ** 2
train_data["GrLivArea-2"] = train_data["GrLivArea"] ** 2
# train_data["SimplOverallQual-s2"] = train_data["SimplOverallQual"] ** 2
train_data["ExterQual-2"] = train_data["ExterQual"] ** 2
train_data["GarageCars-2"] = train_data["GarageCars"] ** 2
train_data["TotalBath-2"] = train_data["TotalBath"] ** 2
train_data["KitchenQual-2"] = train_data["KitchenQual"] ** 2

# Differentiate numerical features and categorical features
categorical_features = train_data.select_dtypes(include = ["object"]).columns
numerical_features = train_data.select_dtypes(exclude = ["object"]).columns
# numerical_features = numerical_features.drop("SalePrice")
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
train_num = train_data[numerical_features]
train_cat = train_data[categorical_features]
print("NAs for numerical features in train : " + str(train_num.isnull().values.sum()))
train_num = train_num.fillna(train_num.median())
print("Remaining NAs for numerical features in train : " + str(train_num.isnull().values.sum()))

#2.6 turn category features to dummy
print("NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))
train_cat = pd.get_dummies(train_cat, drop_first=True)
print("Remaining NAs for categorical features in train : " + str(train_cat.isnull().values.sum()))

#2.7 join categorical and numerical features 
train_Data_New = pd.concat([train_num, train_cat], axis = 1)
print("New number of features : " + str(train_Data_New.shape[1]))

#2.8 remove collinear columns
# Create correlation matrix
corr_matrix = train_Data_New.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
# Drop features 
train_Data_New.drop(to_drop, axis=1, inplace=True)
print(train_Data_New.shape)
train_Data_New.to_csv("2919-159.csv")
print(train_Data_New.shape)


               skewness
Utilities      0.999315
Street         0.995890
Condition2     0.989726
RoofMatl       0.982192
Heating        0.978082
LandSlope      0.946575
CentralAir     0.934932
Functional     0.931507
PavedDrive     0.917808
Electrical     0.914384
LandContour    0.897945
ExterCond      0.878082
SaleType       0.867808
Condition1     0.863014
BldgType       0.835616
SaleCondition  0.820548
MSZoning       0.788356
RoofStyle      0.781507
LotConfig      0.720548
LotShape       0.633562
ExterQual      0.620548
HeatingQC      0.507534
KitchenQual    0.503425
HouseStyle     0.497260
Foundation     0.443151
MSSubClass     0.367123
Exterior1st    0.352740
Exterior2nd    0.345205
MoSold         0.173288
Neighborhood   0.154110
Numerical features : 57
Categorical features : 22
NAs for numerical features in train : 26262
Remaining NAs for numerical features in train : 0
NAs for categorical features in train : 11676
Remaining NAs for categorical features in train : 0
New number of 

In [65]:
##COPYRIGHT from 仲益教育
import sys
# sys.path.append("C://Users//Haipxiang He//desktop//Kaggle//Lecture 2")
#conda install mxltend --channel conda-forge
#pip install package name
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
import numpy as np
from Help_functions_v2 import sklearn_Pvalue, sklearn_adjR2, RMSE

#1: import data
train_data=pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/May19thVer/2919-159.csv")
print(train_data.shape)
X=train_data.iloc[:1460]
print(X.shape)
Test_data=train_data[1460:]
print(Test_data.shape)
Test_data.drop("SalePrice",axis=1,inplace=True)
Test_data.drop("Id",axis=1,inplace=True)


(2919, 161)
(1460, 161)
(1459, 161)
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [66]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Columns: 161 entries, Unnamed: 0 to Heating_Wall
dtypes: float64(17), int64(143), object(1)
memory usage: 3.6+ MB


In [67]:
#2.1: feature selection with p_value
train_data.index=train_data["Id"]
train_data.drop("Id", axis=1, inplace=True)
dependentV=train_data["SalePrice"]
train_data.drop("SalePrice", axis=1, inplace=True)
#Partition the dataset in train + validation sets
#usually linear regression needs at least 30 observations
#split of train and validation can be 70:30, or 60:40
X_train, X_test, y_train, y_test = train_test_split(train_data, dependentV, test_size = 0.3, random_state = 0)
print("X_train : " + str(X_train.shape))
print("X_test : " + str(X_test.shape))
print("y_train : " + str(y_train.shape))
print("y_test : " + str(y_test.shape))

#2.1.1 select top 20 features with the best F-stats
X_scored = SelectKBest(score_func=f_regression, k=20)
X_scored.fit(X_train, y_train)
feat_list=X_scored.get_support()

feature_scoring = pd.DataFrame({
        'feature': X_train.columns[feat_list],
        'pvalue': X_scored.pvalues_[feat_list]
    })

print(feature_scoring)
#homework, find out the RMSE of in sample and out of sample regression of the top 20 selected features

#2.1.2 select all features with individual p_value <=0.05
X_scored2 = SelectKBest(score_func=f_regression, k='all').fit(X_train, y_train)

feature_scoring2 = pd.DataFrame({
        'feature': X_train.columns,
        'pvalue': X_scored2.pvalues_
    })
feat_pvalue_significant=feature_scoring[feature_scoring2.pvalue<=0.05]
feat_pvalue_significant['feature'].values

#2.1.3 select N features based on the 
#2.2 feature selection based on forward/backward elimination based on R Square
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
#2.2.1 find the best k features using stepforward method
stepforward = SFS(LinearRegression(), 
           k_features=10, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=0)
stepforward = stepforward.fit(np.array(X_train), y_train)
print(X_train.columns[list(stepforward.k_feature_idx_)])
#homework, find out the RMSE of in sample and out of sample regression of selected features 
#using forward elimination

#2.2.2 find the best k features using stepbackward method
backward = SFS(LinearRegression(), 
           k_features=10, 
           forward=False, 
           floating=False, 
           verbose=2,
           scoring='r2',
           cv=0)
backward = stepforward.fit(np.array(X_train), y_train)
print(X_train.columns[list(backward.k_feature_idx_)])


X_train : (2043, 159)
X_test : (876, 159)
y_train : (2043,)
y_test : (876,)


ValueError: could not convert string to float: 'x'

In [51]:
#3 feature selection with regularization
#3.1 Ridge Regularization
ridge_model=Ridge(alpha=1)
ridge_model.fit(X_train, y_train)
y_trainPred=ridge_model.predict(X_train)
#in sample
print("in-sample r-squared is")
print(r2_score(y_train, y_trainPred))
print ("RMSE is of in-sample")
print(RMSE(y_trainPred,y_train))
#out of sample
y_testPred=ridge_model.predict(X_test)
print("out-of-sample r-squared is")
print(r2_score(y_test, y_testPred))
print ("RMSE is out-of-sample")
print(RMSE(y_testPred,y_test))



in-sample r-squared is
0.8948120661976421
RMSE is of in-sample
0.09253156330346547
out-of-sample r-squared is
0.8659982252684256
RMSE is out-of-sample
0.10110330408167235
/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=2.42866e-17): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


In [59]:
y_model_preRidge=ridge_model.predict(Test_data)
y_model_prerfc=y_model_preRidge

In [63]:
y_model_prerfc=np.around(y_model_prerfc,0)
prediction=np.array(y_model_prerfc).tolist()
test = pd.read_csv("/Users/chixinning/Desktop/kaggle/KaggleMyOwn/ReferenceCode/house-prices-advanced-regression-techniques/test.csv")
test.insert(1,column="SalePrice",value=prediction)
predict_sub=test.drop(test.iloc[:,2:],axis=1)
predict_sub.head()
predict_sub.shape
predict_sub.to_csv('Home_predictionsLassoTeacher.csv',index=False)

In [ ]:
breakdjflashdf=dfas

In [ ]:
3.2 Lasso Regularization
lasso_model=Lasso(alpha=1)
lasso_model.fit(X_train, y_train)
y_trainPred=lasso_model.predict(X_train)
#in sample
print("lasso in-sample r-squared is")
print(r2_score(y_train, y_trainPred))
print ("Lasso RMSE of in-sample is ")
print(RMSE(y_trainPred,y_train))
#out of sample
y_testPred=lasso_model.predict(X_test)
print("Lasso out-of-sample r-squared is")
print(r2_score(y_test, y_testPred))
print ("Lasso RMSE out-of-sample is")
print(RMSE(y_testPred,y_test))